# Model Building NN

Here we will use Nerual Network to do the prediction.

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

For stocks from different clusters. We will train a model separately.

## NN
In this notebook, I will build a  Prediction Model with NN.

In [2]:
def mse(net, features, labels):
    with torch.no_grad():
        mse = criterion(net(features), labels)
    return mse.item()

In [3]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate, weight_decay=weight_decay) 
    net = net.float()
    for epoch in range(num_epochs):
        for X, y in train_iter:
            l = criterion(net(X.float()), y.float())
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_ls.append(mse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(mse(net, test_features, test_labels))
    return train_ls, test_ls

In [4]:
def get_k_fold_data(k, i, X, y):
    # 返回第i折交叉验证时所需要的训练和验证数据
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat((X_train, X_part), dim=0)
            y_train = torch.cat((y_train, y_part), dim=0)
    return X_train, y_train, X_valid, y_valid

In [5]:
def k_fold(k, X_train, y_train, num_epochs,
           learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = model
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        print('fold %d, train mse %f, valid mse %f' % (i, train_ls[-1], valid_ls[-1]))
    return train_l_sum / k, valid_l_sum / k

In [6]:
def semilogy(x_vals, y_vals, x_label, y_label, x2_vals=None, y2_vals=None,
             legend=None, figsize=(3.5, 2.5)):
    plt.figure(figsize=figsize)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.semilogy(x_vals, y_vals)
    if x2_vals and y2_vals:
        plt.semilogy(x2_vals, y2_vals, linestyle=':')
        plt.legend(legend)
    # plt.show()

In [7]:
for i in tqdm(range(5)):
    data0 = pd.read_csv(f"TrainingData/StockType{i}.csv")
    features = data0.drop(["Target"], axis=1)
    fl = features.columns.to_list()
    # standardize the data
    target = data0[['Target']]
    features = features.apply(lambda x: (x - x.mean()) / (x.std()))
    scaler = MinMaxScaler(feature_range=(-1, 1))
    target["Target"] = scaler.fit_transform(data0['Target'].values.reshape(-1,1))
    # prepare the data
    train_features = torch.tensor(features.values, dtype=torch.float)
    train_labels = torch.tensor(target.Target.values, dtype=torch.float)
    model_sequential = nn.Sequential(
          nn.Linear(train_features.shape[1], 128),
          nn.ReLU(),
          nn.Linear(128, 256),
          nn.ReLU(),
          nn.Linear(256, 256),
          nn.ReLU(),
          nn.Linear(256, 256),
          nn.ReLU(),
          nn.Linear(256, 1)
        )
    class Net(nn.Module):
        def __init__(self, features):
            super(Net, self).__init__()

            self.linear_relu1 = nn.Linear(features, 128)
            self.linear_relu2 = nn.Linear(128, 256)
            self.linear_relu3 = nn.Linear(256, 256)
            self.linear_relu4 = nn.Linear(256, 256)
            self.linear5 = nn.Linear(256, 1)
        
        def forward(self, x):

            y_pred = self.linear_relu1(x)
            y_pred = nn.functional.relu(y_pred)

            y_pred = self.linear_relu2(y_pred)
            y_pred = nn.functional.relu(y_pred)

            y_pred = self.linear_relu3(y_pred)
            y_pred = nn.functional.relu(y_pred)

            y_pred = self.linear_relu4(y_pred)
            y_pred = nn.functional.relu(y_pred)

            y_pred = self.linear5(y_pred)
            return y_pred
    model = Net(features=train_features.shape[1])

    # 使用均方误差作为损失函数
    criterion = nn.MSELoss(reduction='mean')
    k, num_epochs, lr, weight_decay, batch_size = 5, 50, 0.0005, 0, 64

    train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, weight_decay, batch_size)
    print(f"For cluster {i}")
    print('%d-fold validation: avg train mse %f, avg valid mse %f' % (k, train_l, valid_l))
    print("=======================================")

  0%|          | 0/5 [00:00<?, ?it/s]

fold 0, train mse 0.017802, valid mse 0.015028
fold 1, train mse 0.016537, valid mse 0.020090
fold 2, train mse 0.017840, valid mse 0.015196
fold 3, train mse 0.017772, valid mse 0.015331


 20%|██        | 1/5 [05:33<22:15, 333.98s/it]

fold 4, train mse 0.016397, valid mse 0.020714
For cluster 0
5-fold validation: avg train mse 0.017270, avg valid mse 0.017272
fold 0, train mse 0.011414, valid mse 0.007745
fold 1, train mse 0.010156, valid mse 0.012776
fold 2, train mse 0.010744, valid mse 0.010422
fold 3, train mse 0.010186, valid mse 0.012659


 40%|████      | 2/5 [15:25<24:15, 485.27s/it]

...

fold 4, train mse 0.010181, valid mse 0.012591
For cluster 2
5-fold validation: avg train mse 0.010677, avg valid mse 0.010683
fold 0, train mse 0.023156, valid mse 0.015010
fold 1, train mse 0.022659, valid mse 0.017186
fold 2, train mse 0.020361, valid mse 0.026200
fold 3, train mse 0.021114, valid mse 0.023201


 80%|████████  | 4/5 [27:49<06:21, 381.31s/it]

fold 4, train mse 0.020324, valid mse 0.026330
For cluster 3
5-fold validation: avg train mse 0.021523, avg valid mse 0.021586
fold 0, train mse 0.006421, valid mse 0.005031
fold 1, train mse 0.006116, valid mse 0.006304
fold 2, train mse 0.006109, valid mse 0.006280
fold 3, train mse 0.005668, valid mse 0.008046


100%|██████████| 5/5 [43:22<00:00, 520.45s/it]

fold 4, train mse 0.006413, valid mse 0.005063
For cluster 4
5-fold validation: avg train mse 0.006145, avg valid mse 0.006145
